In [ ]:
# Dependency management
try:
    from lxml import etree
except ImportError:
    print("Installing lxml...")
    %pip install lxml==5.3.0
    from lxml import etree  # Re-import after installation

# Standard library imports
import sqlite3
import glob
import os

# Local module imports
from python_cdm_utils.import_sdc_form_sqlite import process_xml
from python_cdm_utils.import_vol_v_message_sqlite import import_data_from_hl7

DB_FILEPATH = "public/sdc_cdm.db"

In [ ]:
if os.path.exists(DB_FILEPATH):
    print(f"Deleting existing database file: {DB_FILEPATH}")
    os.remove(DB_FILEPATH)

# Connect to the SQLite database (creates the database file if it doesn't exist)
conn = sqlite3.connect(DB_FILEPATH)
cursor = conn.cursor()
print("Connected to the SQLite database.")

# Get a list of all .sql files in the ./ddl directory
sql_files = glob.glob('../database/ddl/sqlite/*.sql')

# Sort the files alphanumerically
sql_files.sort()

# Display the files to be executed
print("SQL files to be executed in order:")
for file in sql_files:
    print(os.path.basename(file))

# Execute each SQL file
for sql_file in sql_files:
    print(f"\nExecuting {os.path.basename(sql_file)}...")
    with open(sql_file, 'r') as file:
        sql_script = file.read()
    try:
        cursor.executescript(sql_script)
        print(f"Finished executing {os.path.basename(sql_file)}.")
    except Exception as e:
        print(f"An error occurred while executing {os.path.basename(sql_file)}: {e}")

# Commit changes and close the connection
conn.commit()
conn.close()

print("Database changes committed and connection closed.")

In [ ]:
conn = sqlite3.connect(DB_FILEPATH)
cursor = conn.cursor()

namespaces = {
    "sdc": "urn:ihe:qrph:sdc:2016",
    "xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "xsd": "http://www.w3.org/2001/XMLSchema",
}

# Import SDC templates
v2_messages = glob.glob(os.path.join("../sample_data/sdc_templates", "*.xml"))

# Iterate over each XML file
for message in v2_messages:
    # Read the XML content
    with open(message, "r", encoding="utf-8") as file:
        message_str = file.read()

    tree = etree.parse(message)
    root = tree.getroot()

    form_design = root.find("sdc:FormDesign", namespaces)
    if form_design is None:
        form_design = root

    if form_design is None:
        print(f"No Form Design found in {os.path.basename(message)}")
        continue
    print(f"Form Design: {form_design}")

    # Extract column values from the XML content
    sdcformdesignid = form_design.get("ID")
    baseuri = form_design.get("baseURI")
    lineage = form_design.get("lineage")
    version = form_design.get("version")
    fulluri = form_design.get("fullURI")
    formtitle = form_design.get("formTitle")
    sdc_xml = etree.tostring(form_design).decode("utf-8")
    doctype = "FD"  # TODO: Parse from fullURI

    # Insert the data into the database
    try:
        cursor.execute(
            """
            INSERT INTO main.templatesdcclass (
                sdcformdesignid, baseuri, lineage, version, fulluri, formtitle, sdc_xml, doctype
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """,
            (
                sdcformdesignid,
                baseuri,
                lineage,
                version,
                fulluri,
                formtitle,
                sdc_xml,
                doctype,
            ),
        )
        print(f"Inserted data from {os.path.basename(message)} into the database.")
    except Exception as e:
        print(
            f"An error occurred while inserting data from {os.path.basename(message)}: {e}"
        )

# Commit the changes
conn.commit()
print("All XML data has been inserted into the database.")

cursor.execute("SELECT COUNT(*) FROM main.templatesdcclass;")
count = cursor.fetchone()[0]
print(f"\nTotal records inserted: {count}")

# Commit changes and close the connection
conn.close()

In [ ]:
# Import SDC forms
conn = sqlite3.connect(DB_FILEPATH)
cursor = conn.cursor()

# Import SDC templates
v2_messages = glob.glob(os.path.join("../sample_data/sdc_xml", "*.xml"))

# Iterate over each XML file
for message in v2_messages:
    # Read the XML content
    with open(message, "r", encoding="utf-8") as file:
        message_str = file.read()

    tree = etree.parse(message)
    root = tree.getroot()
    process_xml(root, cursor)

cursor.close()
conn.commit()
conn.close()


In [ ]:
# Import NAACCR data
conn = sqlite3.connect(DB_FILEPATH)
cursor = conn.cursor()

v2_messages = glob.glob(os.path.join("../sample_data/naaccr_v2", "*.hl7"))

# Iterate over each XML file
for message in v2_messages:
    try:
        # Read the XML content
        with open(message, "r", encoding="utf-8") as file:
            message_str = file.read()

        import_data_from_hl7(cursor, message_str)
    except Exception as e:
        print(f"Error processing message: {message}: {e}")
        continue

cursor.close()
conn.commit()
conn.close()